In [1]:
%load_ext lab_black
import pandas as pd
import json
import requests
import numpy as np

from config import g_key


import plotly.express as px

# Part II : VacationPy
----


In [2]:
city_data_to_load = "Resources/city_result.csv"

city_data = pd.read_csv(city_data_to_load)
df = city_data
df.head(2)

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bambous Virieux,-20.34,57.76,75.2,60,40,6.93,MU,1600869990
1,1,Barrow,71.29,-156.79,30.2,68,90,6.93,US,1600869902


In [3]:
df["Lat_Lon"] = df["Lat"].astype(str) + "," + df["Lon"].astype(str)
df.head(2)

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat_Lon
0,0,Bambous Virieux,-20.34,57.76,75.2,60,40,6.93,MU,1600869990,"-20.34,57.76"
1,1,Barrow,71.29,-156.79,30.2,68,90,6.93,US,1600869902,"71.29,-156.79"


In [4]:
perfect_weather_df = df[
    (df["Max Temp"] <= 80)
    & (df["Max Temp"] >= 60)
    & (df["Wind Speed"] <= 20)
    & (df["Cloudiness"] <= 30)
]
perfect_weather_df.head(2)

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Lat_Lon
7,7,Rikitea,-23.12,-134.97,71.92,73,1,13.85,PF,1600869992,"-23.12,-134.97"
8,8,Aksu,41.12,80.26,70.07,25,0,3.09,CN,1600869993,"41.12,80.26"


In [5]:
lat_lon_list = perfect_weather_df["Lat_Lon"].tolist()

In [6]:
hotels = requests.get(
    "https://maps.googleapis.com/maps/api/place/nearbysearch/json",
    params={
        "key": g_key,
        "location": lat_lon_list[17],
        "radius": "5000",
        "type": "lodging",
    },
).json()

In [7]:
hotels

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 41.4113711, 'lng': -73.4152646},
    'viewport': {'northeast': {'lat': 41.4126229802915,
      'lng': -73.4140460697085},
     'southwest': {'lat': 41.4099250197085, 'lng': -73.41674403029151}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'name': 'Courtyard by Marriott Danbury',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 4000,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105731878337185886845">Courtyard by Marriott Danbury</a>'],
     'photo_reference': 'CmRaAAAAf7k5nSQVTHjMicsRDbET-C6e05v7Utw5BNtRh45Dc-tLK9DkcJuas0S8UHe2-3CjT_uZ4ceP4cK9QBLsIFJBDq1QPuF-3E4ZF03XcLm_Sud_ppPR7ThE5Pq2aODqR_ZZEhAQHJPXaMVr-vrwoG3k7ZT6GhR8HcGLFKtT10ijv_Znpdsw7P2mtQ',
     'width': 5993}],
   'place_id': 'ChIJFUu_ndf-54kRPT9YW753kqA',
   'plus_code': {'compound_code': 'CH6M+GV Danbury, CT, USA',
    'global_code':

In [8]:
def extract_data(data):
    data = data["results"][0]
    return {
        "Hotel Name": data["name"],
        "lat": data["geometry"]["location"]["lat"],
        "lon": data["geometry"]["location"]["lng"],
    }

In [9]:
hotel_results = []
for lat_lon in lat_lon_list:
    try:
        print(lat_lon)
        data = requests.get(
            "https://maps.googleapis.com/maps/api/place/nearbysearch/json",
            params={
                "key": g_key,
                "location": lat_lon,
                "radius": "5000",
                "type": "lodging",
            },
        ).json()
        hotel_result = extract_data(data)
        hotel_results.append(hotel_result)
    except Exception as e:
        print(e)
        print(f"{lat_lon} Not found. Skipping...")
        pass

-23.12,-134.97
41.12,80.26
57.97,33.25
42.1,-79.24
-34.42,19.23
-16.48,-151.75
43.48,-8.24
46.87,42.06
-33.93,18.42
-15.2,12.15
-21.2,-175.2
-25.24,130.99
-19.55,169.27
-33.65,115.33
-33.02,27.91
-34.53,20.04
-8.93,125.41
41.37,-73.41
47.38,28.82
-26.65,15.16
-25.4,-70.48
28.66,-17.92
-31.4,26.36
-29.62,-54.18
59.7,30.79
-33.59,26.89
-32.55,115.7
-35.33,-72.42
-33.01,17.94
9.3,13.4
-28.55,16.43
-28.78,32.04
54.17,17.49
-30.18,-50.21
37.61,-122.49
-23.25,-50.98
30.48,-115.95
40.66,-73.58
56.89,21.19
17.96,-102.2
20.89,-156.47
42.91,73.27
59.78,30.82
37.1,-113.58
-22.96,14.51
28.12,-16.73
35.15,-114.57
44.51,50.26
-14.83,-64.9
26.59,127.98
-6.6,147.87
47.57,26.2
-34.0,24.73
-25.86,26.9
-19.84,34.84
-20.28,44.28
-33.92,25.57
-13.85,136.42
-17.73,-149.15
41.14,-73.36
43.75,-81.72
45.32,55.2
30.37,-88.56
42.56,-114.46
31.55,-110.3
41.71,-70.23
32.68,109.02
-20.74,116.85
54.08,21.38
-20.73,139.5
8.27,-80.55
-17.53,-149.33
31.26,117.28
-28.97,-51.07
10.93,108.1
50.93,8.4
-20.23,164.02
55.15,2

In [10]:
hotel_results

[{'Hotel Name': "Pension Maro'i", 'lat': -23.1206197, 'lon': -134.9687529},
 {'Hotel Name': 'Pudong Holiday Hotel',
  'lat': 41.14176,
  'lon': 80.29139599999999},
 {'Hotel Name': 'Valdayskiye Zori, Gostinichno-Restorannyy Kompleks',
  'lat': 57.97428599999999,
  'lon': 33.268093},
 {'Hotel Name': 'Hampton Inn & Suites Jamestown',
  'lat': 42.1184574,
  'lon': -79.2427728},
 {'Hotel Name': 'Misty Waves Boutique Hotel',
  'lat': -34.4213898,
  'lon': 19.2366413},
 {'Hotel Name': 'Bora-Bora Pearl Beach Resort & Spa',
  'lat': -16.4759407,
  'lon': -151.7724793},
 {'Hotel Name': 'Carrís Almirante', 'lat': 43.486307, 'lon': -8.2302042},
 {'Hotel Name': 'ТеТ-А-ТеТ', 'lat': 46.8537973, 'lon': 42.0573134},
 {'Hotel Name': 'Southern Sun Waterfront Cape Town',
  'lat': -33.9157633,
  'lon': 18.4234956},
 {'Hotel Name': 'Hotel Chik Chik Namibe',
  'lat': -15.1978317,
  'lon': 12.1575544},
 {'Hotel Name': 'Keleti Beach Resort',
  'lat': -21.2036689,
  'lon': -175.2324594},
 {'Hotel Name': 'Desert

In [11]:
hotel_results_df = pd.DataFrame.from_dict(hotel_results)
hotel_results_df

,Hotel Name,lat,lon
0,Pension Maro'i,-23.120620,-134.968753
1,Pudong Holiday Hotel,41.141760,80.291396
2,"Valdayskiye Zori, Gostinichno-Restorannyy Komp...",57.974286,33.268093
3,Hampton Inn & Suites Jamestown,42.118457,-79.242773
4,Misty Waves Boutique Hotel,-34.421390,19.236641
...,...,...,...
77,"ŠVIESA, svečių namai, UAB LAZDYNO RIEŠUTAS",55.132678,26.159317
78,Veranda Paul et Virginie Hotel & Spa,-20.006569,57.668077
79,Wedgewood Motel,42.148391,-72.305276
80,Holiday Inn Express Roseville-St. Paul,45.022670,-93.193242
